In [ ]:
#Load 131k seq data 

from Bio import SeqIO
import pandas as pd

eyed = []
name = []
desc = []

for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    eyed.append(seq_record.id)
    desc.append(seq_record.description.lower())

#All following anomalies in this cell had species name missing
#Fix anomaly at index 21244

desc[21244] = 'MK293717 |Gemycircularvirus strain RN/Haikou/China| complete genome|complete|China|Rattus norvegicus|Gemycircularvirus'.lower()

#Fix anomaly at index 40853

desc[40853] = 'mf407276 |bacillus phage vb_bvep-goe6| complete genome|complete||bacillus velezensis fzb42|bacillus phage'

#Fix anomaly at index 50896

desc[50896] = 'kx349291 |cyanophage s-rim44 isolate es_42_0910| complete genome|complete|usa|synechococcus|cyanophage'

#Fix other cyanophage anomalies

c = 0
l = []
for i in desc:
    if i[-1] == '|':
        l.append(c)
    c+=1
    
for i in l:
    desc[i] = desc[i] + 'cyanophage' 

In [ ]:
#Species name missing detection
c = 0
l = []
for i in desc:
    if i[-1] == '|':
        print(i, c)
    c+=1

In [ ]:
for i in l:
    print(desc[i])

In [ ]:
#load sequence id and description into dataframe

df = pd.DataFrame({'Sequence description' : desc})
df['ID'] = eyed

In [ ]:
#Get species name by taking last '|' split section from description

l = [x.split('|')[-1].lower() for x in df['Sequence description']]
df['Species'] = l

In [ ]:
#New dataframe with only unique species names

df2 = df.drop_duplicates(subset = 'Species')

#leaves 17046 rows

In [ ]:
#Shorten species name upto 'virus' for viruses and upto 'phage' for phages

tl = list(df2['Species'])
tl2 = []

for i in tl:
    if 'virus' in i:
        tl2.append(i.split('virus')[0] + 'virus')
    elif 'phage' in i:
        tl2.append(i.split('phage')[0] + 'phage')
    else:
        tl2.append(i)
        
df2.loc[:, 'Species'] = tl2

In [ ]:
#checking which entries have species names starting with phage or virus

# c = 0
# for i in tl2:
#     if i == 'virus':
#         print(tl[c])
#     elif i == "phage":
#         print(tl[c])
#     c+=1

In [ ]:
#if you want to look at the the weridest species names

# for i in df2['Species'].tolist():
#     if 'virus' not in i and 'phage' not in i and 'viroid' not in i and 'viridae' not in i and 'satellite' not in i:
#         print(i)

In [ ]:
#Drop duplicates on Species again

df2 = df2.drop_duplicates(subset = 'Species').reset_index(drop= True)

#remove just 'virus' at index 688
#100% make sure this code does not run twice
df2 = df2.drop(axis = 0, index = 688).reset_index(drop= True)

#leaves 4818 rows

#Making new 'Genome type' column

df2['Genome type'] = 'dunno'

End of primary filtering for species in bulk sequence data file
Start of genome type labelling

Using ViRAD .txt files 

In [ ]:
#loading files into lists p, n and d

f = open('plus ssRNA.txt', 'r')
p = f.readlines()
f.close()
f = open('minus ssRNA.txt', 'r')
n = f.readlines()
f.close()
f = open('dsRNA.txt', 'r')
d = f.readlines()
f.close()

In [ ]:
#join p, n and d into three large strings

p2 = ''
p2 = p2.join(p).lower()
n2 = ''
n2 = n2.join(n).lower()
d2 = ''
d2 = d2.join(d).lower()

Anomaly detection

In [ ]:
#found both in p2 and n2

for i in df2['Species'].tolist():
    if i in p2 and i in n2:
        print('wtf', i, df2['Species'].tolist().index(i))

##Reveals 3 anomalies. output: wtf cosavirus 174 \n wtf virus 688 \n wtf  2403
#Hardcoded species name to species name missing values
#Cosavirus is probably plus ssRNA. Manually annotated in cell below
#hardcoded removal of just 'virus' from df2 in cell above

In [ ]:
#found both in p2 and d2

for i in df2['Species'].tolist():
    if i in p2 and i in d2:
        print('wtf', i, df2['Species'].tolist().index(i))
        
        
#gives output wtf pseudomonas phage 27 \n wtf phage 49 \n wtf rna virus 305
#d2 has 5 pseudomonas phage strains and p2 has 2. Hoping to manually hardcode them below
#rna virus comes from 'rna virus hoopoe/bbanka01/2015/hun' which has unknown genome. hardlabelling
df2.loc[df2['Species'] == 'rna virus', 'Species'] = 'rna virus hoopoe'
df2.loc[df2['Species'] == 'rna virus hoopoe', 'Genome type'] = 'unknown RNA'

# phage 49 will be hard to fix for reasons hard to explain rn. Will ignore for now.

In [ ]:
#found both in n2 and d2

for i in df2['Species'].tolist():
    if i in n2 and i in d2:
        print('wtf', i, df2['Species'].tolist().index(i))
        
        
#gives output: wtf yam virus 1017
#yam virus y is probably an unknown RNA virus. Skimming the paper did not reveal genome type. The family Betaflexiviridae is
# probably a plus RNA family, so manually labelling as plus RNA

In [ ]:
#label

c = 0 

for i in df2['Species'].tolist():
    if i in p2:
        df2.loc[c, 'Genome type'] = 'plus ssRNA'
    elif i in n2:
        df2.loc[c, 'Genome type'] = 'minus ssRNA'
    elif i in d2:
        df2.loc[c, 'Genome type'] = 'dsRNA'
    c+=1
                
##Reveals 3 anomalies. output: wtf cosavirus 174 | wtf virus 688 | wtf  2403
#Hardcoded 2403 species name
#Cosavirus is probably 'plus ssRNA'. Manually annotating
df2.loc[174, 'Genome type'] = 'plus ssRNA'
#hardcoded removal of just virus from df2 in cell above
#hardcoding 'yam virus' as 'plus ssRNA' 
df2.loc[1017, 'Genome type'] = 'plus ssRNA'

In [ ]:
#hardcoding 5 + 2 = 7 pseudomonas phages

df3 = df.drop_duplicates(subset= 'Species').reset_index(drop = True)

pl = []
c = 0
for i in tl2:
    if i == 'pseudomonas phage':
        pl.append(c)
    c+=1
        
for i in pl:
    if df3.loc[i, 'Species'] in p2:
        nd = dict(df3.loc[i, :])
        nd['Genome type'] = 'plus ssRNA'
        df2 = df2.append(nd, ignore_index=True)
    elif df3.loc[i, 'Species'] in d2:
        nd = dict(df3.loc[i, :])
        nd['Genome type'] = 'dsRNA'
        df2 = df2.append(nd, ignore_index=True)
        
#this adds 1 new plus ssRNA value and 2 new dsRNA values

In [ ]:
df2.loc[df2['Genome type'] == 'minus ssRNA']

In [ ]:
len(df2.loc[df2['Genome type'] == 'dsRNA'])

Code above labels 579 rows as 'plus ssRNA' and 39 rows as 'minus ssRNA' and 85 as 'dsRNA' and 1 as 'unknown RNA'

4116 rows left unlabelled

End of using the three ViRAD files.

Data could be labelled by matching the family/order names in those files. Will explore later

Starting the use of NCBI taxonomy and ICTV file

In [6]:
#Loading data

df3 = pd.read_csv('tax.csv')
df4 = pd.read_csv('ICTV 2018b.v2.csv')

In [7]:
#Take first comma separated part of 'Selected lineage' column as a 'Fam' column

df3['Fam'] = [x.split(',')[0] for x in df3['Selected lineage'].to_list()]

In [8]:
#Make a new dataframe df5 which relates a Family with its Genome type

df5 = pd.DataFrame({'Fam' : df4['Family'], 'Genome type' : df4['Genome Composition']})
df5 = df5.drop_duplicates(subset = 'Fam')

#merge df3 to df5 to get an expected Genome type for every NCBI entry

df3 = pd.merge(df3, df5, how = 'left', on = 'Fam')

#Out of 212830 entries, only 4028 remain unlabelled
#Only 15008 out of 212k entries have a unique taxonomy name

In [123]:
# for some reason 'Neighbor' in df3 and 'ID' in df have no common entries.
# downloading data from ncbi (not ncbi virus) , that share the same accession number scheme

In [124]:
#trying to merge tax data with 131k ncbi seq data

In [125]:
df2 = df2.reset_index(drop = True)

In [126]:
df3 = df3.drop_duplicates(subset = 'Taxonomy name').reset_index(drop = True)
df3['Taxonomy name'] = [x.lower() for x in df3['Taxonomy name'].to_list()]

In [129]:
dft = df3[['Taxonomy name', 'Fam', 'Genome type']].copy().reset_index(drop = True)

In [130]:
dft = dft.rename(columns = {'Taxonomy name' : 'Species'})

In [131]:
dft = pd.merge(df2, dft, on = 'Species', how= 'left')

In [132]:
dft.loc[dft['Genome type'] == 'ssRNA(-)']

,Sequence description,ID,Species,Fam,Genome type
109,mn832721 |crimean-congo hemorrhagic fever orth...,MN832721,crimean-congo hemorrhagic fever orthonairovirus,Nairoviridae,ssRNA(-)
181,mn850091 |andes orthohantavirus strain arch14/...,MN850091,andes orthohantavirus,Hantaviridae,ssRNA(-)
324,mn264268 |oropouche virus strain ov/homo sapie...,MN264268,oropouche orthobunyavirus,Peribunyaviridae,ssRNA(-)
359,mk395271 |porcine respirovirus 1 strain gd2018...,MK395271,porcine respirovirus 1,Paramyxoviridae,ssRNA(-)
371,mn233932 |rabies lyssavirus strain nl.2012.062...,MN233932,rabies lyssavirus,Rhabdoviridae,ssRNA(-)
...,...,...,...,...,...
16217,fj905105 |lyssavirus ozernoe| complete genome|...,FJ905105,lyssavirus ozernoe,Rhabdoviridae,ssRNA(-)
16426,ef646763 |prospect hill virus segment l| compl...,EF646763,prospect hill orthohantavirus,Hantaviridae,ssRNA(-)
16523,ef533944 |hantavirus z10 segment s| complete s...,EF533944,hantavirus z10,Hantaviridae,ssRNA(-)
16570,dq284451 |cano delgadito virus strain vhv-574 ...,DQ284451,cano delgadito orthohantavirus,Hantaviridae,ssRNA(-)


3830 rows labelled

exporting 125*2 rows

In [133]:
dfx = pd.concat([dft.loc[dft['Genome type'] == 'ssRNA(-)'], dft.loc[dft['Genome type'] == 'ssRNA(+)'].reset_index(drop = True).loc[:124]])

In [134]:
dfx = dfx[['Sequence description', 'Species', 'Genome type']].copy().reset_index(drop = True)

In [3]:
dfx = pd.read_csv('fib.csv')

In [4]:
dexes = []
for i in dfx['Sequence description'].to_list():
    dexes.append(desc.index(i))

In [178]:
dexes = [int(i) for i in dexes]

In [19]:
#something impossible happens here

sex = []
c = 0
for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    for i in dexes:
        if c == i:
            sex.append(seq_record.seq)
    c+=1
print(len(sex))

In [22]:
dfx['Seq'] = sex

In [23]:
dfy = dfx[['Species', 'Genome type', 'Seq']].copy()

In [25]:
dfy.to_csv('fib2.csv', index = False)

In [ ]:
len(sex)

In [151]:
dex2 = []
for x, y in sex:
    dex2.append(y)

In [160]:
dex2.sort()

In [167]:
len(sex)

249

In [155]:
for i in dexes:
    if i not in dex2:
        print(i)

In [113]:
dfx.to_csv('fib.csv', index=False)

In [ ]:
#df3['Taxonomy name'] = df3['Taxonomy name'].lower()

In [ ]:
'Y15034' in df3['Representative']

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "mysticmacrophage@gmail.com"
with Entrez.efetch(
db="nucleotide", rettype="gb", retmode="text", id="A04321"
) as handle:
    seq_record = SeqIO.read(handle, "gb")
    print("%s with %i features" % (seq_record.description, len(seq_record.features)))

In [ ]:
df3.sort_values(by='Neighbor')

In [ ]:
'A04321' in df['ID'].to_list()

In [ ]:
len(df3.loc[df3['Genome type'].isnull()])

In [ ]:
ndf = pd.read_csv('first test label.csv')
ndf = ndf[~ndf['Genome type'].isin(['dunno'])].reset_index(drop=True)
ndf = ndf.drop(axis = 1, labels = 'Unnamed: 0')

In [ ]:
l = []

for i in ndf['Sequence description']:
    l.append(desc.index(i))

In [ ]:
c = 0
for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    if c in l:
        l.append(seq_record.seq)
    c+=1

In [6]:
l = []
c = 0
for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    if c in dexes:
        l.append(seq_record.seq)
    c+=1

In [ ]:
import numpy as np
a = np.array(l)
a = a.reshape(704, 2, order = 'F')

In [ ]:
ndf['Seq'] = l[704:]

In [ ]:
ndf.to_csv('first test label.csv', index = False)

In [ ]:
l[-5]

In [ ]:
str(l[-5].translate(11))

In [ ]:
from os.path import join
from os import listdir, rmdir
from shutil import move

root = r'C:\Users\Michael\Downloads\_'
# for filename in listdir(join(root, '_')):
#     move(join(root, '_', filename), join(root, filename))
# rmdir(root)

In [ ]:
listdir(join(root, 'abaca_bunchy_top_virus_uid28697'))

In [ ]:
from Bio import SeqIO
import pandas as pd

eyed = []
name = []
desc = []

for seq_record in SeqIO.parse('sequence (17).gb', "genbank"):
    eyed.append(seq_record.id)
print(len(eyed))

In [ ]:
import pandas as pd 
x = pd.read_csv('sequence.seq')

In [ ]:
for i in x:
    if x in df['']